In [1]:
# Loading the Gender Bias Model for accessing the word cluster
import warnings
warnings.filterwarnings("ignore")


from LanguageBiasesInReddit import DADDBias_ICWSM

# Model trained on our 300k+ comments from Reddit
modelpath = "model/DADDBiasResModel_redditALL"

ts1 = ["sister" , "female" , "woman" , "girl" , "daughter" , "she" , "hers" , "her"]
ts2   = ["brother" , "male" , "man" , "boy" , "son" , "he" , "his" , "him"]  


# Getting biased wordd
b1,b2 = DADDBias_ICWSM.GetTopMostBiasedWords(
    modelpath,                         
    300,                               
    ts1,                               
    ts2,                               
    ['JJ','JJR','JJS'],                
    True)                              

# Making clusters
cl1,cl2 = DADDBias_ICWSM.Cluster(
  b1,                                
  b2,                                
  0.15,                              
  100                                
)


[nltk_data] Error loading averaged_perceptron_tagger: <urlopen error
[nltk_data]     [Errno 11001] getaddrinfo failed>
[nltk_data] Error loading vader_lexicon: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>


...0...100...200...300...400...500...600...700...800...900...1000...1100...1200...1300...1400New partition for ts1, intrasim:  0.19495323565826128
New partition for ts2, intrasim:  0.08447111857697098
New partition for ts1, intrasim:  0.15723658681870903
New partition for ts2, intrasim:  0.0897217252655286
New partition for ts1, intrasim:  0.19086260746428185
New partition for ts2, intrasim:  0.08916755933100681
New partition for ts1, intrasim:  0.16081789310562564
New partition for ts2, intrasim:  0.06840458485238948
New partition for ts1, intrasim:  0.20125664365700718
New partition for ts2, intrasim:  0.10368102961798849
New partition for ts1, intrasim:  0.19963875475500678
New partition for ts2, intrasim:  0.055594709692722365
New partition for ts1, intrasim:  0.15567752252548445
New partition for ts2, intrasim:  0.09226624095173676
New partition for ts1, intrasim:  0.16219489642227045
New partition for ts2, intrasim:  0.11570792094441676
New partition for ts1, intrasim:  0.2108550

In [2]:
# Calculate centroid of all the clusters
import numpy as np


def cluster_centroid(cluster:list):
    b_c = []
    cl_c = []
    for cl in cluster:
        wvecs = np.array([x["wv"] for x in cl])
        b_c.append((np.array([x["bias"] for x in cl])).mean())
        res = np.zeros(200)
        for wv in wvecs:
            res = np.add(res, wv)
        
        res /= len(wvecs)
        cl_c.append(res)

    return (b_c, cl_c)

b_centroid1, cl_centroids1 = cluster_centroid(cl1)
b_centroid2, cl_centroids2 = cluster_centroid(cl2)


In [4]:
# This is how we crate embeddings for new words, using the Glove model
# We will use the vector of the new word and try to see the cosine similarity with other cluster centroids
# For instance Im generating a 200 dimensional word embedding for the word "hardworking"
import gensim.downloader as api
model = api.load("glove-wiki-gigaword-200")
test_word = "hardworking"
test_embed = np.zeros(200)

if test_word in model.vocab:
    test_embed = model[test_word] 

test_embed

array([ 3.7826e-01, -6.8909e-01,  5.3033e-01,  1.8311e-01,  2.9940e-03,
       -3.1450e-01, -2.7197e-01,  7.5340e-01,  4.3112e-01,  2.6673e-01,
        4.1256e-01,  5.4872e-03, -4.8211e-03,  2.4733e-01,  2.1620e-03,
        5.4634e-01,  1.5450e-01, -5.4535e-01, -2.4168e-01, -5.4336e-02,
       -3.2885e-02, -4.8395e-02, -7.5751e-01, -3.1743e-01, -1.9289e-01,
        1.9384e-01, -1.2811e-02, -2.1807e-02,  3.0757e-01, -2.5669e-01,
        2.3886e-01,  4.1901e-01, -1.0066e-02,  5.5867e-01, -1.6383e-01,
       -4.9719e-01, -8.2963e-02, -8.5484e-01, -2.7994e-02,  8.3389e-01,
        3.8190e-01,  4.1880e-01,  1.9130e-01, -3.3851e-01, -4.1635e-01,
       -1.5544e-01, -6.7855e-01,  2.4623e-01, -3.0660e-01,  4.2270e-02,
       -7.1387e-01,  4.8535e-02, -1.3877e-01,  1.8556e-01, -1.7320e-01,
       -4.3011e-01,  2.6577e-01, -1.2328e-01, -1.2237e-01, -1.1380e+00,
        5.9151e-02, -1.4690e-01, -5.6969e-01,  1.0009e-01, -1.2276e-01,
        4.6944e-01,  6.0160e-01,  5.2718e-01, -4.3816e-01, -5.98

In [95]:
# loading our test data
import pandas as pd

testing_data = pd.read_csv("data/test_data.csv")
testing_data = testing_data[["Username", "Subreddit", "Comment"]]

sample_sz = 100
testing_sample = testing_data[testing_data.Subreddit == "relationships"].sample(sample_sz)
testing_sample

,Username,Subreddit,Comment
792203,luv3ly,relationships,"Yikes OP, you may have ruined something very s..."
12541,TrustMyOpinion,relationships,"No offense, but your message sort of comes acr..."
168173,Omega037,relationships,"Well, try to learn a lesson about infatuation,..."
641061,Noble_toaster,relationships,Money is the most important thing in this worl...
114992,motorsizzle,relationships,Because he's trying to stay friendly with the ...
...,...,...,...
1088858,kintu,relationships,Have a feeling that you did not post a lot of ...
302715,shattered_wavelength,relationships,"For me, kissing is kinda... filler. It's what ..."
1093861,secretxletters,relationships,This is so sad. You should divorce him either ...
10784,Omega037,relationships,It's anonymous and addressed to her. I suppos...


In [88]:
# defining some utility functions
from scipy import spatial


def cosine_similarity(embed1, embed2):
    return spatial.distance.cosine(embed1, embed2)


def create_sentence_embedding(sentence: list[str]):
    sentence_embeds = []

    for word in sentence:
        word_embedding = np.zeros(200)
        if word in model.vocab:
            word_embedding = model[word]
        sentence_embeds.append(word_embedding)

    return sentence_embeds


def calculate_sentence_bias(sentence_embeds):
    msentence_bias = 0
    fsentence_bias = 0


    for i, wv in enumerate(sentence_embeds):
        compare_cl1 = []
        compare_cl2 = []

        for cluster1 in cl_centroids1:
            compare_cl1.append(cosine_similarity(wv, cluster1))
        for cluster2 in cl_centroids2:
            compare_cl2.append(cosine_similarity(wv, cluster2))
        # print(max(compare_cl1), max(compare_cl2))
        # print(np.argmax(compare_cl1), np.argmax(compare_cl2))
        msentence_bias += b_centroid2[np.argmax(compare_cl2)]
        msentence_bias += b_centroid1[np.argmax(compare_cl1)]

    return (msentence_bias, msentence_bias)


In [100]:
m_biases = []
f_biases = []
gender_target = []
removed = []

for i, c in enumerate(testing_sample.Comment):
    s = str(c).split(" ")
    if (len(s)) < 5:
        removed.append(i)
        continue
    
    sembed = create_sentence_embedding(s)
    mbias, fbias = calculate_sentence_bias(sembed)
    m_biases.append(mbias)
    f_biases.append(fbias)
    # if mw_count < fw_count : gender_target.append(1)
    # else : gender_target.append(0)


In [101]:
after_valid = testing_sample.copy(deep=True)
after_valid.index = np.arange(sample_sz)
after_valid.drop(index=removed, axis=0, inplace=True)

after_valid["Male Bias"] = m_biases
after_valid["Female Bias"] = f_biases
after_valid["Bias - Diff"] = [x-y for x,y in zip(f_biases, m_biases)]

after_valid.head(40)


,Username,Subreddit,Comment,Male Bias,Female Bias,Bias - Diff
0,luv3ly,relationships,"Yikes OP, you may have ruined something very s...",4.398348,5.444474,1.046126
1,TrustMyOpinion,relationships,"No offense, but your message sort of comes acr...",1.494593,2.035471,0.540878
2,Omega037,relationships,"Well, try to learn a lesson about infatuation,...",2.358940,2.684448,0.325507
3,Noble_toaster,relationships,Money is the most important thing in this worl...,1.745452,2.157113,0.411661
4,motorsizzle,relationships,Because he's trying to stay friendly with the ...,2.178491,2.684643,0.506152
5,zuesk134,relationships,&gt; He is afraid I am going to see something...,3.096590,3.486930,0.390340
6,AliceA,relationships,I'd give her the benefit of a high stress situ...,1.841561,2.358337,0.516776
7,kintu,relationships,Not to rub it in but you kept making stupid de...,2.136306,3.146877,1.010570
8,RestlessBrowSyndrome,relationships,"I dont know when she updated, but this is in t...",6.503668,8.805433,2.301765
9,zuesk134,relationships,i think filing for sole custody could actually...,3.623621,4.412499,0.788878


In [105]:
after_valid.iloc[13].Comment

"I completely understand.  It pisses me off so much when my wife doesn't use her psychic powers when she is sleeping.\n\nYou need to have a serious talk with her.  Let her know that if she wants to continue to date a over-entitled man-child, she really needs to step up her psychic game.  And if that means no more sleeping, then so be it. That's what I did and after a full week of not sleeping, my wife learned how to use her powers while sleeping.  The only downside is I used to just wait till she was asleep to cheat on her.  Now I don't get to at all anymore.\n\nNow as for your other question\n&gt; Is this what destroys relationships?\n\nYour behavior is.  Not her's."

In [98]:
max(after_valid["Bias - Diff"])

0.7319355334952282

In [99]:
pd.DataFrame.to_csv(after_valid, "Relationships_subreddit_BIGSENT_Analysis.csv")

In [49]:
testing_sample.iloc[23].Comment


'Woah. Did that just happen? A regulation win? '

In [50]:
after_valid.Targeted_to.value_counts()

1    191
0      9
Name: Targeted_to, dtype: int64

In [ ]:
# defining some utility functions
from scipy import spatial


def cosine_similarity(embed1, embed2):
    return spatial.distance.cosine(embed1, embed2)


def create_sentence_embedding(sentence: list[str]):
    sentence_embeds = []

    for word in sentence:
        word_embedding = np.zeros(200)
        if word in model.vocab:
            word_embedding = model[word]
        sentence_embeds.append(word_embedding)

    return sentence_embeds


def calculate_sentence_bias(sentence_embeds):
    msentence_bias = []
    fsentence_bias = []

    for i, wv in enumerate(sentence_embeds):
        compare_cl1 = []
        compare_cl2 = []

        for cluster1 in cl_centroids1:
            compare_cl1.append(cosine_similarity(wv, cluster1))
        for cluster2 in cl_centroids2:
            compare_cl2.append(cosine_similarity(wv, cluster2))
        # print(max(compare_cl1), max(compare_cl2))
        # print(np.argmax(compare_cl1), np.argmax(compare_cl2))
        if max(compare_cl1) < max(compare_cl2):
            msentence_bias.append(b_centroid2[np.argmax(compare_cl2)])
        else:
            fsentence_bias.append(b_centroid1[np.argmax(compare_cl1)])

    if msentence_bias == []:
        m_bias = 0
    else:
        m_bias = max(msentence_bias)
        
    if fsentence_bias == []:
        f_bias = 0
    else:
        f_bias = max(fsentence_bias)

    return (m_bias, f_bias)
